<a href="https://colab.research.google.com/github/conextm/python/blob/main/Validacion_cruzada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, f_classif

**Predicción de la Intención de Compra de Usuarios en Amazon**

Amazon quiere mejorar la precisión con la que predice si un usuario, después de navegar por el sitio, realizará una compra dentro de las próximas 24 horas. Esto permitirá a Amazon ofrecer ofertas personalizadas a tiempo o hacer recomendaciones específicas para incentivar la compra.

**Características del Dataset:** Historial de navegación, categorías de productos visitados, tiempo en el sitio, historial de compras previas, frecuencia de visitas en el último mes, reviews realizadas, entre otras.

**Variable Objetivo**: Compra (1 si el usuario compra dentro de las próximas 24 horas, 0 en caso contrario).

In [ ]:
df = pd.read_csv('fictional_amazon_products.csv')

In [ ]:
df.head(1)

,Unnamed: 0,historial_navegacion,tiempo_en_sitio,visitas_ultimo_mes,categoria_producto,historial_compras,reviews,edad,membresia_prime,dias_desde_ult_compra,Compra
0,0,52,30.377761,7,libros,baja,positiva,32,1,39,1


In [ ]:
# --- Preprocesamiento y Modelado ---
X = df.drop('Compra', axis=1)
y = df['Compra']

num_features = ['historial_navegacion', 'tiempo_en_sitio', 'visitas_ultimo_mes', 'edad', 'membresia_prime', 'dias_desde_ult_compra']
cat_features = ['categoria_producto', 'historial_compras', 'reviews']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(), cat_features)
    ])

kbest = SelectKBest(f_classif, k=5)  # Selecciona las 5 mejores características, pero puedes ajustar 'k' según lo necesites

# Modelo Logístico
logistic_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selector', kbest),
    ('classifier', LogisticRegression())
])

# Árbol de Decisión
tree_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selector', kbest),
    ('classifier', DecisionTreeClassifier())
])

In [ ]:
# Validación cruzada con 10 folds
logistic_scores = cross_val_score(logistic_pipeline, X, y, cv=10, scoring='accuracy')
tree_scores = cross_val_score(tree_pipeline, X, y, cv=10, scoring='accuracy')

print(f"Exactitud promedio con Regresión Logística y SelectKBest: {logistic_scores.mean():.3f}")
print(f"Exactitud promedio con Árboles de Decisión y SelectKBest: {tree_scores.mean():.3f}")

Exactitud promedio con Regresión Logística y SelectKBest: 0.866
Exactitud promedio con Árboles de Decisión y SelectKBest: 0.997
